In [0]:
%run "./config"

In [0]:
spark.conf.set(
    "fs.azure.account.key.container.dfs.core.windows.net",
    f'{key}'
)

In [0]:
df_sales=spark.read.parquet(f"{processed_folder_path}/salesdata", header= True, inferSchema= True)

In [0]:
df_product=spark.read.parquet(f"{processed_folder_path}/productdata", header= True, inferSchema= True)

In [0]:
df_join= df_sales.join(df_product, df_sales.ProductID == df_product.ProductID)\
                  .drop(df_product.ProductID,df_product.Date )\
                  .drop('SalesOrderDetailID', 'UnitPriceDiscount')\
                  .withColumnRenamed('Product Name','ProductName')\
                  .withColumnRenamed('Category Name','CategoryName')
df_join.show(2)

+------------+--------+---------+---------+---------+----------+---------+-----------+------------+---------+------+----+--------------------+
|SalesOrderID|OrderQty|ProductID|UnitPrice|LineTotal| Salesdate|     Date|ProductName|StandardCost|ListPrice|Weight|Size|        CategoryName|
+------------+--------+---------+---------+---------+----------+---------+-----------+------------+---------+------+----+--------------------+
|       71784|      10|      707|   20.994|   209.94|2008-06-01|2025-3-21|    Helmets|     13.0863|    34.99|  NULL|NULL|Sport-100 Helmet,...|
|       71783|      10|      707|   20.994|   209.94|2008-06-01|2025-3-21|    Helmets|     13.0863|    34.99|  NULL|NULL|Sport-100 Helmet,...|
+------------+--------+---------+---------+---------+----------+---------+-----------+------------+---------+------+----+--------------------+
only showing top 2 rows


In [0]:
#save as file
final_data= df_join.write.mode( "overwrite").parquet(f"{presentation_folder_path}/finaldata")

In [0]:
#save as external table
final_data_table= df_join.write.format('delta').mode( "overwrite").option("path", "abfss://presentation@container.dfs.core.windows.net/tables").saveAsTable("final_data_table")

In [0]:
%sql
--save as internal table
create database if not exists sales;
create table if not exists sales.salesorderdetail


In [0]:
df_join.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("sales.salesorderdetail")


In [0]:
# Write to SQL
jdbcUrl = "jdbc:sqlserver://mystudent.database.windows.net:1433;databaseName=mysqlstudent"
connProperties = {
  "user": "XXXXX",
  "password": "XXXXX",
  "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

df_join.write \
    .mode("overwrite") \
    .jdbc(url=jdbcUrl, table="salesdata", properties=connProperties)
